# Vidurkiai duomenų analitikoje: geometrinis, harmoninis ir svertinis

Šiame faile pateikiami trys dažni vidurkiai, kurie praverčia analizuojant **augimą**, **rodiklius „per vienetą“** ir **svertinius KPI**.
Naudojami paprasti verslo pavyzdžiai ir minimalūs duomenų rinkiniai, kad rezultatai būtų lengvai patikrinami.

**Naudojamos bibliotekos:**
- `numpy` – skaičiavimams ir svertiniam vidurkiui
- `scipy.stats` – geometriniam ir harmoniniam vidurkiui (patogu ir aišku)
- `pandas` – duomenų lentelėms ir tvarkingam rezultatų pateikimui


In [ ]:
import numpy as np
import pandas as pd

# SciPy funkcijos yra patogios, nes aiškiai išreiškia ketinimą (gmean/hmean).
from scipy.stats import gmean, hmean

## 1) Geometrinis vidurkis (Geometric Mean)

**Kada tinka:**
- kai analizuojamas **augimas per laiką** (pvz., mėnesiniai augimo koeficientai, investicijų grąžos daugikliai)
- kai procesas yra **multiplikatyvus** (dauginami koeficientai)

**Kodėl ne aritmetinis vidurkis:**
- aritmetinis vidurkis neteisingai apibendrina augimą, kai pokyčiai yra procentiniai ir kinta stipriai
- geometrinis vidurkis geriau atspindi **„tipinį“ daugiklį** per laikotarpį

Toliau pateikiamas pavyzdys su mėnesiniais pardavimų pokyčiais.


In [3]:
# Verslo situacija: mėnesiniai pardavimų pokyčiai (multiplikatoriai).
# Pavyzdžiui, +10% reiškia daugiklį 1.10, -5% reiškia 0.95 ir pan.

growth_factors = np.array([1.10, 0.95, 1.20, 0.90, 1.05])

geo = gmean(growth_factors)
geo

1.0345366218871734

Geometrinis vidurkis grąžina **vidutinį augimo daugiklį**. Dažnai norima jį pateikti procentais.

In [4]:
avg_growth_pct = (geo - 1) * 100
avg_growth_pct

3.4536621887173435

### Patikra per sudėtinį efektą

Jeigu per kelis mėnesius taikomi augimo daugikliai, bendras pokytis gaunamas juos sudauginus. Geometrinis vidurkis pakeltas laipsniu (mėnesių skaičius) turi atitikti bendrą daugiklį.


In [7]:
overall_factor = growth_factors.prod().round(2)
overall_factor

1.19

In [6]:
# Sudėtinis efektas per tą patį laikotarpį, naudojant geometrinį vidurkį:
reconstructed_overall = geo ** len(growth_factors)
overall_factor, reconstructed_overall

(1.1850299999999998, 1.1850299999999998)

### Dažnos klaidos ir gerosios praktikos (geometrinis vidurkis)

**Dažnos klaidos**
- **Maišomi procentai ir daugikliai:** vietoje 1.10 įrašoma 10 (tada rezultatas tampa beprasmis).
- **Neigiamos arba nulinės reikšmės:** geometrinis vidurkis klasikine forma reikalauja **teigiamų** skaičių.
- **Trūkstamos reikšmės (NaN):** būtina tvarkingai apdoroti prieš skaičiavimą.

**Gerosios praktikos**
- Augimą reprezentuoti **daugikliais** (`1 + pct_change`).
- Prieš skaičiavimą patikrinti, ar visos reikšmės yra **> 0**.
- Aiškiai komunikuoti, kad rezultatas yra **daugiklis**, o ne procentai.


## 2) Harmoninis vidurkis (Harmonic Mean)

**Kada tinka:**
- kai skaičiuojami rodikliai **„per vienetą“** (pvz., Eur/vnt, km/h, užklausos/sek.)
- kai svarbu „teisingai“ apibendrinti santykius, o didelės reikšmės neturi dominuoti

Toliau pateikiamas pavyzdys su **kainomis už vienetą** pas skirtingus tiekėjus, kai norima apibendrinti tipinę kainą.


In [13]:
unit_prices = np.array([2.50, 2.60, 2.40, 2.55, 123.20])  # Eur/vnt

arith = unit_prices.mean()
harm = hmean(unit_prices).round(2)

arith, harm

(26.65, 3.12)

Aritmetinis vidurkis dažniau „pakeliamas“ dėl didesnių kainų. Harmoninis vidurkis dažnai būna mažesnis ir geriau tinka, kai domina tipinis **Eur/vnt** santykis.

### Klasikinis pavyzdys: greitis

Kai važiuojama tą patį atstumą skirtingais greičiais, teisingas „vidutinis greitis“ yra harmoninis, o ne aritmetinis.


In [17]:
# Važiuojama tą patį atstumą dviem atkarpomis: 60 km/h ir 90 km/h.
speeds = np.array([60, 90])

arith_speed = speeds.mean()
harm_speed = hmean(speeds)

arith_speed, harm_speed

(75.0, 72.0)

### Dažnos klaidos ir gerosios praktikos (harmoninis vidurkis)

**Dažnos klaidos**
- **Nulis** duomenyse: harmoninis vidurkis su 0 yra neapibrėžtas (dalyba iš nulio).
- **Neigiamos reikšmės:** praktiniuose verslo rodikliuose „per vienetą“ dažniausiai neturi prasmės.
- Harmoninis vidurkis taikomas ten, kur tinka aritmetinis (pvz., tiesioginės sumos, o ne santykiai).

**Gerosios praktikos**
- Naudoti harmoninį vidurkį tik santykiams „per vienetą“.
- Prieš skaičiavimą patikrinti, ar visi skaičiai yra **> 0**.
- Jei duomenys turi apimtis (kiekiai, laikai), dažnai reikia **svertinės** versijos (žr. svertinis vidurkis).


## 3) Svertinis vidurkis (Weighted Mean)

**Kada tinka:**
- kai reikšmės turi skirtingą **svorį** (apimtį, svarbą, kiekį)
- kai skaičiuojamas, pavyzdžiui, **vidutinis pardavimo tarifas** ar **vidutinė kaina** pagal parduotus kiekius

Toliau pateikiamas pavyzdys: skirtingos prekės parduodamos skirtingais kiekiais, todėl teisinga vidutinė kaina turi būti **svertinė**.


In [25]:
df = pd.DataFrame({
    "Product": ["A", "B", "C", "D"],
    "UnitPrice": [10.0, 12.0, 8.0, 20.0],   # Eur/vnt
    "Quantity":  [100,  20,  200,  5]       # vnt
})

df

,Product,UnitPrice,Quantity
0,A,10.0,100
1,B,12.0,20
2,C,8.0,200
3,D,20.0,5


In [26]:
# Aritmetinis vidurkis ignoruoja kiekius (Quantity).
arith_price = df["UnitPrice"].mean()

# Svertinis vidurkis įvertina, kad daugiausia parduota ten, kur Quantity didesnis.
weighted_price = np.average(df["UnitPrice"], weights=df["Quantity"]).round(2)

arith_price, weighted_price

(12.5, 9.05)

### Praktinė interpretacija

Svertinis vidurkis atsako į klausimą: **kokia yra vidutinė kaina, kai kiekviena kaina sveriama parduotu kiekiu**.
Tai atitinka bendrą pajamų ir bendro kiekio santykį.


In [27]:
revenue = (df["UnitPrice"] * df["Quantity"]).sum()
total_qty = df["Quantity"].sum()

ratio_price = revenue / total_qty
weighted_price, ratio_price

(9.05, 9.046153846153846)

### Dažnos klaidos ir gerosios praktikos (svertinis vidurkis)

**Dažnos klaidos**
- Netinkami svoriai (pvz., naudojamas `UnitPrice` kaip svoris vietoje `Quantity`).
- Neigiami svoriai arba svoriai su `NaN` (dažnai reiškia prastą duomenų kokybę).
- Svertinis vidurkis skaičiuojamas, kai svoriai nėra palyginami (pvz., skirtingų matų vienetai tame pačiame stulpelyje).

**Gerosios praktikos**
- Svorius rinktis pagal verslo logiką: kiekiai, apimtys, klientų skaičius, darbo valandos, portfelio dalis.
- Prieš skaičiavimą patikrinti: ar svoriai yra **>= 0** ir ar nėra trūkstamų reikšmių.
- Rezultatą validuoti per formulę: **(sum(value * weight) / sum(weight))**.


## Mini santrauka: kada kurį vidurkį rinktis

- **Geometrinis vidurkis** tinka **augimo koeficientams** ir **sudėtiniam efektui**.
- **Harmoninis vidurkis** tinka rodikliams **„per vienetą“** (kaina/vnt, greitis, našumas).
- **Svertinis vidurkis** tinka, kai reikšmės turi skirtingas **apimtis** arba **svarbą**.

Papildoma pastaba: jei santykiai „per vienetą“ turi apimtis (pvz., kainos per skirtingus kiekius), dažniausiai reikia **svertinio** požiūrio.
